In [1]:
import os, sys, time, importlib, osmium, logging, pyproj
import osmnx

import geopandas as gpd
import pandas as pd
import networkx as nx
import numpy as np
import shapely.wkb as wkblib

from functools import partial
from shapely.ops import transform
from shapely.geometry import Point, LineString

# import current version of GOSTnets
sys.path.insert(0,"../")

import GOSTnets as gn
import GOSTnets.load_osmium as losm
import GOSTnets.mapbox_traffic as mb


In [2]:
# Test osmium load to generate G
importlib.reload(gn)
importlib.reload(losm)
osm_file = r"C:\temp\dhaka-latest.osm.pbf"

# Test various attempts to read in G object

In [3]:
%%time
G = losm.create_G(osm_file, verbose=False, project='', densify = False)

Wall time: 11.6 s


In [ ]:
print(G.number_of_edges())
print(G.number_of_nodes())

In [ ]:
%%time
G2 = losm.create_G(osm_file, verbose=False, project=3106, densify = False)

In [ ]:
print(G2.number_of_edges())
print(G2.number_of_nodes())

In [ ]:
%%time
G3 = losm.create_G(osm_file, verbose=False, project='', densify = True)

In [ ]:
print(G3.number_of_edges())
print(G3.number_of_nodes())

In [ ]:
%%time
G4 = losm.create_G(osm_file, verbose=False, project=3106, densify = True)

In [ ]:
print(G4.number_of_edges())
print(G4.number_of_nodes())

# Read in and process mapbox data

In [4]:
# Test processing of mapbox traffic files
mapbox_traffic_file = "C:/temp/1320222-Asia-Dhaka.csv"
mb_traffic = mb.simplify_mapbox_csv(mapbox_traffic_file)

In [5]:
mb_traffic.head()

,FROM_NODE,TO_NODE,min_speed,max_speed,mean_speed
0,1049113953,1128728733,30.0,30.0,30.000000
1,1049114208,1049113755,13.0,13.0,13.000000
2,113675716,264026291,12.0,20.0,15.059028
3,113676779,3443056298,14.0,14.0,14.000000
4,1175962443,1175962451,43.0,67.0,55.340774


In [10]:
gn.example_node(G)

(60916997, {'shape': <shapely.geometry.point.Point object at 0x0000020B5A808AC8>, 'x': 90.3961558, 'y': 23.7408755})


In [9]:
gn.example_edge(G)

(60916997, 6785381369, {'osm_id': 723435766, 'Wkt': <shapely.geometry.linestring.LineString object at 0x0000020B6029B5F8>, 'length': 0.0017781345121973288, 'infra_type': 'service', 'osm_nodes': <shapely.geometry.linestring.LineString object at 0x0000020B6029B5F8>})


In [ ]:
nodes_df = gn.node_gdf_from_graph(G)

Skipped due to missing geometry data: (5638848886, {})
Skipped due to missing geometry data: (5643048453, {})
Skipped due to missing geometry data: (4869524298, {})
Skipped due to missing geometry data: (4869524299, {})
Skipped due to missing geometry data: 

In [ ]:
# Generate reference to the edges and node datasets from the network
edges_df = gn.edge_gdf_from_graph(G, geometry_tag='Wkt')


In [16]:
gn.edge_gdf_from_graph?

In [13]:
xx

OutMultiEdgeView([(60916997, 6785381369, 0), (60917111, 387810798, 0), (60917472, 264894197, 0), (60917472, 1720308798, 0), (60917472, 2320154898, 0), (60917544, 3341431962, 0), (60917561, 4823690506, 0), (60917561, 4823690508, 0), (61742180, 61748178, 0), (61742182, 61745128, 0), (61742187, 4899286160, 0), (61742187, 322074775, 0), (61742189, 61742264, 0), (61742196, 61745144, 0), (61742199, 61746929, 0), (61742201, 61746933, 0), (61742204, 61746937, 0), (61742206, 4976029374, 0), (61742208, 61742211, 0), (61742211, 61742208, 0), (61742255, 259006184, 0), (61742255, 61742257, 0), (61742257, 322074775, 0), (61742257, 61742255, 0), (61742264, 61742189, 0), (61742267, 322074901, 0), (61742306, 61746969, 0), (61742311, 322074757, 0), (61742311, 3444099543, 0), (61745070, 61745118, 0), (61745070, 61748178, 0), (61745118, 61745070, 0), (61745118, 61748173, 0), (61745123, 3444123723, 0), (61745128, 61742182, 0), (61745144, 61742196, 0), (61745144, 61746929, 0), (61746929, 61742199, 0), (6174